`python3 -m pip install -U pandas plotly nbformat`

`pip install "https://github.com/DCMLab/wavescapes/archive/refs/heads/johannes.zip"`

In [1]:
%reload_ext autoreload
%autoreload 2
import numpy as np
from wavescapes import normalize_dft

from etl import display_wavescapes, get_human_analyses, get_metadata, get_pcms, test_dict_keys, make_feature_vectors
from utils import max_pearsonr_by_rotation, pitch_class_matrix_to_tritone, get_from_to, get_all_from_to

## Settings

In [2]:
import os
DEBUSSY_REPO = '.'
DATA_FOLDER = '~/debussy_figures/data'
#DATA_FOLDER = './data/data'
WAVESCAPE_FOLDER = '~/debussy_figures/wavescapes'
wavescape_folder = os.path.relpath(os.path.expanduser(WAVESCAPE_FOLDER), os.getcwd())
EXAMPLE_FNAME = 'l108_morceau'
LONG_FORMAT = True

## Loading metadata
Metadata for all pieces contained in the dataset.

In [3]:
metadata = get_metadata(DEBUSSY_REPO)
metadata.columns

Metadata for 82 files.


Index(['rel_paths', 'last_mc', 'last_mn', 'length_qb', 'length_qb_unfolded',
       'all_notes_qb', 'n_onsets', 'TimeSig', 'KeySig', 'label_count',
       'composer', 'workTitle', 'movementNumber', 'movementTitle',
       'workNumber', 'poet', 'lyricist', 'arranger', 'copyright',
       'creationDate', 'mscVersion', 'platform', 'source', 'translator',
       'musescore', 'ambitus', 'comment', 'comments', 'composed_end',
       'composed_start', 'originalFormat', 'pdf', 'staff_1_ambitus',
       'staff_1_instrument', 'staff_2_ambitus', 'staff_2_instrument',
       'staff_3_ambitus', 'staff_3_instrument', 'transcriber', 'typesetter',
       'year', 'median_recording', 'qb_per_minute', 'sounding_notes_per_qb',
       'sounding_notes_per_minute'],
      dtype='object')

## Loading precomputed summary wavescape

Colour legend for most resonant coefficients:

![legend](legend.png)

In [4]:
display_wavescapes(wavescape_folder, EXAMPLE_FNAME, norm_method=4, summaries=True, rows=1)

,


## Loading human analysis data

In [5]:
analyses = get_human_analyses(DEBUSSY_REPO)
analyses.tail()

,L,mc_start,mc_start_offset,mc_end,mc_end_offset,structure,transposition,comments,source,fname,from_qb,to_qb
244,75-03,35.0,0,37.0,0,modal,Db aeolian,NaN,SL,l075-03_suite_clair,153.0,162.0
245,75-03,37.0,0,41.0,0,majmin,E maj,NaN,SL,l075-03_suite_clair,162.0,180.0
246,75-03,43.0,0,47.0,0,modal,Ab mixolydian,NaN,SL,l075-03_suite_clair,189.0,207.0
247,108,11.0,0,15.0,0,wt,0,pure,JH,l108_morceau,20.0,28.0
248,108,15.0,0,19.0,0,wt,1,pure,JH,l108_morceau,28.0,36.0


## Example for getting from quarterbeat (qb) indications to matrix indices
EXAMPLE_FNAME = 'l108_morceau', mc_start = 15, mc_end = 19, --> `from_qb = 28`, `to_qb = 36`

segment length = 8 --> `x = 7`

last included qb = 35 --> `y = 35`

In [6]:
test = get_pcms(DEBUSSY_REPO, long=False)[EXAMPLE_FNAME]
test[7, 35] # this is what we expect the function get_from_to() to return

array([0.  , 4.75, 0.  , 6.25, 0.  , 6.  , 0.  , 2.25, 0.  , 4.5 , 0.  ,
       3.75])

In [7]:
pcms = get_pcms(DEBUSSY_REPO, long=LONG_FORMAT)
get_from_to(pcms[EXAMPLE_FNAME], 28, 36, long=LONG_FORMAT)

array([0.  , 4.75, 0.  , 6.25, 0.  , 6.  , 0.  , 2.25, 0.  , 4.5 , 0.  ,
       3.75])

Likewise, we can get the vectors for all triangles in a particular interval.

For the given example, this long format result contains:

* index 0-7: pitch class vectors for each quarter in mm. 15-19
* index 8-14: 7 half note windows
* 15-20: 6 windows of a dotted half note
* 21-25: 5 whole note windows
* 26-29: 4 windows of length 5 quarters
* 30-32: 3 windows of a dotted whole note
* 33-34: 2 windows of length 7 quarters
* 35: top node (same as above)

In [8]:
get_all_from_to(pcms[EXAMPLE_FNAME], 28, 36, long=LONG_FORMAT)

array([[0.  , 0.75, 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 1.5 , 0.  ,
        0.75],
       [0.  , 1.5 , 0.  , 0.75, 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ,
        0.75],
       [0.  , 0.  , 0.  , 1.5 , 0.  , 1.5 , 0.  , 0.  , 0.  , 0.  , 0.  ,
        0.  ],
       [0.  , 1.  , 0.  , 1.  , 0.  , 2.25, 0.  , 0.75, 0.  , 1.5 , 0.  ,
        0.  ],
       [0.  , 0.  , 0.  , 0.  , 0.  , 0.75, 0.  , 1.5 , 0.  , 0.  , 0.  ,
        0.75],
       [0.  , 0.  , 0.  , 1.5 , 0.  , 0.75, 0.  , 0.  , 0.  , 0.75, 0.  ,
        0.  ],
       [0.  , 0.75, 0.  , 0.75, 0.  , 0.75, 0.  , 0.  , 0.  , 0.  , 0.  ,
        0.75],
       [0.  , 0.75, 0.  , 0.75, 0.  , 0.  , 0.  , 0.  , 0.  , 0.75, 0.  ,
        0.75],
       [0.  , 2.25, 0.  , 0.75, 0.  , 0.  , 0.  , 0.  , 0.  , 1.5 , 0.  ,
        1.5 ],
       [0.  , 1.5 , 0.  , 2.25, 0.  , 1.5 , 0.  , 0.  , 0.  , 0.  , 0.  ,
        0.75],
       [0.  , 1.  , 0.  , 2.5 , 0.  , 3.75, 0.  , 0.75, 0.  , 1.5 , 0.  ,
        0.  ],
       [0.  , 1.  , 0

# Creating 9-fold vectors for prototypes

* 6 normalized DFT magnitudes
* maximal correlation with major profile from Mozart piano sonatas
* maximal correlation with minor profile from Mozart piano sonatas
* tritone detector

In [9]:
def pcs2array(pcs):
    """Pass a collection of pitch classes between 0-11 to get a (1, 12) pitch class matrix,
    where the first row is a pitch class vector in which every present PC is 1."""
    pcs = set(pcs)
    assert all(0 <= i <= 11 for i in pcs), "pitch classes need to be within [0, 11]"
    result = [int(i in pcs) for i in range(12)]
    return np.array([result])


pentatonic_pcm = pcs2array([0,2,4,7,9])
pentatonic_pcm

array([[1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0]])

In [10]:
norm_params = ('0c', True)

def make_ninefold(pcm, norm_params):
    """Pass a (n, 12) pitch class matrix to get the corresponding 9-fold vectors for each row."""
    how, indulge_prototypes = norm_params
    dft_coeffs = np.fft.fft(pcm)[..., :7]
    norm_mag_phase = normalize_dft(dft_coeffs, how=how, indulge_prototypes=indulge_prototypes)
    mags = norm_mag_phase[...,0]
    result = np.column_stack([
        mags,
        max_pearsonr_by_rotation(pcm, 'mozart_major'),
        max_pearsonr_by_rotation(pcm, 'mozart_minor'),
        pitch_class_matrix_to_tritone(pcm)
    ])
    return result


make_ninefold(pentatonic_pcm, norm_params)

array([[0.05358984, 0.2       , 0.2       , 0.4       , 0.74641016,
        0.2       , 0.75965026, 0.6264067 , 0.        ]])

In [11]:
wholetone_pcm = pcs2array([0,2,4,6,8,10])
wholetone_pcm

array([[1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0]])

In [12]:
make_ninefold(wholetone_pcm, norm_params)

array([[0.00000000e+00, 3.70074342e-17, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 1.00000000e+00, 1.03515767e-02, 1.49650089e-01,
        2.88675135e-01]])

In [13]:
major_pcm = pcs2array([0,2,4,5,7,9,11])
make_ninefold(major_pcm, norm_params)

array([[0.03827846, 0.14285714, 0.14285714, 0.14285714, 0.53315012,
        0.14285714, 0.84626241, 0.67922096, 0.14285714]])

## Loading pickled 9-fold vectors

Then we can analyse a human annotation by comparing the 9-fold vectors for all comprised triangles with each of the prototypical 9-fold vectors.

In [14]:
norm_params = ('0c', True)
ninefold_dict = make_feature_vectors(DATA_FOLDER, norm_params=norm_params, long=LONG_FORMAT)
test_dict_keys(ninefold_dict, metadata)
ninefold_dict[EXAMPLE_FNAME].shape

Found matrices for all files listed in metadata.tsv.


(1431, 9)

In [15]:
get_all_from_to(ninefold_dict[EXAMPLE_FNAME], 28, 36, LONG_FORMAT)

array([[ 0.6830127 ,  0.25      ,  0.5       ,  0.25      ,  0.6830127 ,
         1.        ,  0.60942677,  0.50470706,  0.        ],
       [ 0.75      ,  0.25      ,  0.        ,  0.25      ,  0.75      ,
         1.        ,  0.52412408,  0.46427977,  0.        ],
       [ 0.8660254 ,  0.5       ,  0.        ,  1.        ,  0.8660254 ,
         1.        ,  0.51633132,  0.53355565,  0.        ],
       [ 0.29738657,  0.11538462,  0.46153846,  0.19230769,  0.29738657,
         1.        ,  0.37694134,  0.53938713,  1.67705098],
       [ 0.5       ,  0.5       ,  0.5       ,  1.        ,  0.5       ,
         1.        ,  0.53485075,  0.5779131 ,  0.5625    ],
       [ 0.4330127 ,  0.75      ,  0.        ,  1.        ,  0.4330127 ,
         1.        ,  0.38425076,  0.3640446 ,  1.125     ],
       [ 0.4330127 ,  0.25      ,  0.        ,  0.25      ,  0.4330127 ,
         1.        ,  0.37573639,  0.36064395,  0.5625    ],
       [ 0.4330127 ,  0.25      ,  0.        ,  0.25      ,  0